# Алгоритмы Витерби

In [1]:
import numpy as np

In [58]:
def open_state_to_index(state):
    mapping = {'О': 0, 'Р': 1}
    return mapping[state]

def index_to_hidden_state(idx):
    mapping = [1, 2]
    return mapping[idx]

In [59]:
def viterbi(open_states, P, A, B):
    assert len(P) == A.shape[0] == B.shape[0]
    psi = []
    idx = open_state_to_index(open_states[0])
    epsilon = (P * B[:, idx]).reshape(-1, 1)
    for i, open_state in enumerate(open_states[1:]):
        idx = open_state_to_index(open_state)
        psi.append(np.argmax(A * epsilon, axis=0))
        epsilon = (np.max(A * epsilon, axis=0) * B[:, idx]).reshape(-1, 1)
    hidden_states = []
    hidden_states.append(np.argmax(epsilon))
    n = len(psi)
    for i in range(n):
        hidden_states.append(psi[n - i - 1][hidden_states[i - 1]])
    hidden_states.reverse()
    return [index_to_hidden_state(idx) for idx in hidden_states]

# Алгоритм Forward-Backward

In [147]:
def forward_backward(open_states, P, A, B):
    assert len(P) == A.shape[0] == B.shape[0]
    n = len(open_states)
    alphas = []
    idx = open_state_to_index(open_states[0])
    alpha = (P * B[:, idx]).reshape(1, -1)
    alphas.append(alpha)
    for i, open_state in enumerate(open_states[1:]):
        idx = open_state_to_index(open_state)
        alpha = (alpha @ (A * B[:, idx].reshape(1, -1))).reshape(1, -1)
        alphas.append(alpha)
    alphas = np.array(alphas).squeeze()
    
    betas = []
    beta = np.ones_like(P).reshape(-1, 1)
    for i in range(n):
        idx = open_state_to_index(open_states[n - i - 1])
        beta = ((A * B[:, idx].reshape(-1, 1)) @ beta).reshape(-1, 1)
        betas.append(beta)
    betas.reverse()
    betas = np.array(betas).squeeze()
    probs = alphas * betas
    probs[:, 0] /= (alphas * betas).sum(axis=1)
    probs[:, 1] /= (alphas * betas).sum(axis=1)
    return probs

## Тестирование

In [148]:
open_states = 'ОРОРОРООРРРРРРРРРРОООООООО'

In [149]:
P = np.array([0.5, 0.5], dtype=np.float128)
P

array([0.5, 0.5], dtype=float128)

## Тест 1

In [150]:
B = np.array(
    [
        [0.5, 0.5],
        [0.1, 0.9]
    ], dtype=np.float128
)
B

array([[0.5, 0.5],
       [0.1, 0.9]], dtype=float128)

In [151]:
A = np.array(
    [
        [0.8, 0.2],
        [0.2, 0.8]
    ], dtype=np.float128
)
A

array([[0.8, 0.2],
       [0.2, 0.8]], dtype=float128)

In [152]:
viterbi(open_states, P, A, B)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [153]:
forward_backward(open_states, P, A, B)

array([[0.96835726, 0.03164274],
       [0.64533767, 0.35466233],
       [0.97303856, 0.02696144],
       [0.66247696, 0.33752304],
       [0.975122  , 0.024878  ],
       [0.68315411, 0.31684589],
       [0.98074602, 0.01925398],
       [0.96846735, 0.03153265],
       [0.31859247, 0.68140753],
       [0.17274253, 0.82725747],
       [0.11519081, 0.88480919],
       [0.09130111, 0.90869889],
       [0.08256203, 0.91743797],
       [0.08275657, 0.91724343],
       [0.09201997, 0.90798003],
       [0.11695831, 0.88304169],
       [0.17701972, 0.82298028],
       [0.33006042, 0.66993958],
       [0.97551682, 0.02448318],
       [0.99271764, 0.00728236],
       [0.99561575, 0.00438425],
       [0.99612883, 0.00387117],
       [0.99618335, 0.00381665],
       [0.99598096, 0.00401904],
       [0.99477011, 0.00522989],
       [0.98788325, 0.01211675]], dtype=float128)

In [154]:
list(np.argmax(forward_backward(open_states, P, A, B), axis=1) + 1)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]

## Тест 2

In [155]:
B = np.array(
    [
        [0.5,  0.5],
        [0.51,  0.49]
    ], dtype=np.float128
)
B

array([[0.5 , 0.5 ],
       [0.51, 0.49]], dtype=float128)

In [156]:
A = np.array(
    [
        [0.5, 0.5],
        [0.5, 0.5]
    ], dtype=np.float128
)
A

array([[0.5, 0.5],
       [0.5, 0.5]], dtype=float128)

In [157]:
viterbi(open_states, P, A, B)

[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]

In [158]:
forward_backward(open_states, P, A, B)

array([[0.49009998, 0.50990002],
       [0.51009998, 0.48990002],
       [0.49009998, 0.50990002],
       [0.51009998, 0.48990002],
       [0.49009998, 0.50990002],
       [0.51009998, 0.48990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.51009998, 0.48990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002],
       [0.49009998, 0.50990002]], dtype=float128)

In [159]:
list(np.argmax(forward_backward(open_states, P, A, B), axis=1) + 1)

[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]